In [1]:
lat=12.9716
long=77.5946
print('The geograpical coordinates of Chandigarh are {}, {}.'.format(lat,long))

The geograpical coordinates of Chandigarh are 12.9716, 77.5946.


In [2]:
!pip -q install folium
import folium
banglore_map = folium.Map(location = [lat,long], zoom_start = 10)
folium.Marker([lat,long]).add_to(banglore_map)
#banglore_map.save("maps/Chandigarh Map.html")
banglore_map

In [3]:
CLIENT_ID = 'LBTVKAUHJ02FZAJ4PXSX1MFFNU2NK0QZ0FFNU4UDJOJLZVJS' # your Foursquare ID
CLIENT_SECRET = 'WV1N2KMTI3ELNLSINDDJ4PNDVBMLXNGVLVNGF3HDPJCILOW0' # your Foursquare Secret
VERSION = '20180605'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LBTVKAUHJ02FZAJ4PXSX1MFFNU2NK0QZ0FFNU4UDJOJLZVJS
CLIENT_SECRET:WV1N2KMTI3ELNLSINDDJ4PNDVBMLXNGVLVNGF3HDPJCILOW0


In [4]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors

from pandas.io.json import json_normalize
import requests
RADIUS = 30000
NO_OF_VENUES = 100
pd.set_option('display.max_rows', None)

offset = 0
total_venues = 0
foursquare_venues = pd.DataFrame(columns = ['name', 'categories', 'lat', 'lng'])

while (True):
    url = ('https://api.foursquare.com/v2/venues/explore?client_id={}'
           '&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}').format(CLIENT_ID, 
                                                                        CLIENT_SECRET, 
                                                                        VERSION, 
                                                                        lat, 
                                                                        long, 
                                                                        RADIUS,
                                                                        NO_OF_VENUES,
                                                                        offset)
    result = requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radius of {} Km".format(venues_fetched, RADIUS/1000))

    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)

    # Filter the columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    venues = venues.loc[:, filtered_columns]

    # Filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis = 1)

    # Clean all column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    foursquare_venues = pd.concat([foursquare_venues, venues], axis = 0, sort = False)
    
    if (venues_fetched < 100):
        break
    else:
        offset = offset + 100

foursquare_venues = foursquare_venues.reset_index(drop = True)
print(url)
print("\nTotal {} venues fetched".format(total_venues))

Total 100 venues fetched within a total radius of 30.0 Km
Total 66 venues fetched within a total radius of 30.0 Km
https://api.foursquare.com/v2/venues/explore?client_id=LBTVKAUHJ02FZAJ4PXSX1MFFNU2NK0QZ0FFNU4UDJOJLZVJS&client_secret=WV1N2KMTI3ELNLSINDDJ4PNDVBMLXNGVLVNGF3HDPJCILOW0&v=20180605&ll=12.99635979382966,77.5946&radius=30000&limit=100&offset=100

Total 166 venues fetched


In [51]:
results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
from pandas.io.json import json_normalize

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Shiva's Chats,Snack Place,12.941623,77.560610
1,Wonderla Amusement Park,Theme Park,12.939804,77.401225
2,Kanti Sweets,Bakery,12.911081,77.551690
3,Domino's Pizza,Pizza Place,13.053972,77.507553
4,Barbeque Nation,Multicuisine Indian Restaurant,12.841846,77.646654


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

66 venues were returned by Foursquare.


In [30]:
bang_df=nearby_venues

In [34]:
map_bang = folium.Map(location=[lat,long], zoom_start=11)
for lat, lng,name,categories in zip(bang_df['lat'],bang_df['lng'],bang_df['name'],bang_df['categories']):
    label = '{}, {}'.format(name,categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],radius=5,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.7).add_to(map_bang) 
map_bang

In [54]:
print('There are {} uniques categories.'.format(len(bang_df['categories'].unique())))

There are 40 uniques categories.


In [55]:
bang_df.groupby('categories').count()


,name,lat,lng
categories,,,
American Restaurant,1,1,1
Asian Restaurant,2,2,2
Bakery,1,1,1
Bar,1,1,1
Beer Garden,1,1,1
Bowling Alley,1,1,1
Breakfast Spot,1,1,1
Café,7,7,7
Coffee Shop,3,3,3
